In [9]:
%load_ext autoreload
%autoreload 2

from mint.model import Transformer
from mint.config import config, to_dict
from mint.trainer import Trainer, trainer_config
from datasets import load_from_disk
from transformers import AutoTokenizer
import os
from mint.translator import Translator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
DATASET_PATH = "../preprocessed_dataset"

dataset = load_from_disk(os.path.join(DATASET_PATH, "paired_tokenized_dataset"))
source_tokenizer = AutoTokenizer.from_pretrained(os.path.join(DATASET_PATH, "source_tokenizer"))
target_tokenizer = AutoTokenizer.from_pretrained(os.path.join(DATASET_PATH, "target_tokenizer"))
source_tokenizer.pad_token = "<pad>"
target_tokenizer.pad_token = "<pad>"

In [10]:
config.model.d_model = 512
config.model.n_heads = 1
config.model.max_seq_len = 128
config.model.d_feedforward = 2048
config.model.p_dropout = 0.1
config.model.context_window = None

config.encoder.n_blocks = 10
config.encoder.vocab_size = 10000
config.decoder.n_blocks = 10
config.decoder.vocab_size = 10000

model = Transformer(**to_dict(config))

print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

Number of parameters: 126471680


In [ ]:
trainer_config.logger_config.experiment_name = "exp2"
trainer_config.warmup_steps = 3000
trainer_config.learning_rate = 1e-4
trainer = Trainer(model, dataset, **to_dict(trainer_config))
trainer.train(10)

Logging to /home/mharag/fit/zpja/runs/exp2


avg loss: 9.2853:   0%|          | 15/28140 [00:04<2:22:17,  3.29it/s]

In [96]:
translator = Translator(model, source_tokenizer, target_tokenizer)

translator.translate("hello", max_length=128)

['mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm<|endoftext|>']